## Imports

In [ ]:
import numpy as np
import tensorflow as tf                # Tensor operations and GPU/TPU support
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# Hugging Face Transformers
from transformers import AutoTokenizer,DataCollatorWithPadding, TFAutoModelForSequenceClassification, create_optimizer, pipeline
from transformers.keras_callbacks import KerasMetricCallback
# Datasets & evaluation
from datasets import load_dataset      # 🤗 Datasets loader
import evaluate                        # Unified metrics API


C:\Users\sergi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\sergi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


## Data Preprocessing

**Dataset:** HateBR from 🤗 Datasets
[https://huggingface.co/datasets/ruanchaves/hatebr](https://huggingface.co/datasets/ruanchaves/hatebr)

HateBR comprises Brazilian‑Portuguese political comments scraped from Instagram and labeled for offensive language and hate speech.

* **Size:** 7,000 annotated documents
* **Annotation layers:**

  1. Offensive language — offensive vs. non‑offensive
  2. Offensiveness level — slight, moderate, severe
  3. Hate‑speech category — xenophobia, racism, homophobia, sexism, religious intolerance, political partisanship, dictatorship praise, antisemitism, fatphobia

🤗 Datasets handles download and caching via a single command.
The resulting `DatasetDict` includes the standard splits (e.g., `train`, `validation`, `test`), each exposing typed features and a row count.

Hugging Face Datasets Hub: [https://huggingface.co/datasets](https://huggingface.co/datasets)


In [ ]:
db = load_dataset("ruanchaves/hatebr")
db

DatasetDict({
    train: Dataset({
        features: ['instagram_comments', 'offensive_language', 'offensiveness_levels', 'antisemitism', 'apology_for_the_dictatorship', 'fatphobia', 'homophobia', 'partyism', 'racism', 'religious_intolerance', 'sexism', 'xenophobia', 'offensive_&_non-hate_speech', 'non-offensive', 'specialist_1_hate_speech', 'specialist_2_hate_speech', 'specialist_3_hate_speech'],
        num_rows: 4480
    })
    validation: Dataset({
        features: ['instagram_comments', 'offensive_language', 'offensiveness_levels', 'antisemitism', 'apology_for_the_dictatorship', 'fatphobia', 'homophobia', 'partyism', 'racism', 'religious_intolerance', 'sexism', 'xenophobia', 'offensive_&_non-hate_speech', 'non-offensive', 'specialist_1_hate_speech', 'specialist_2_hate_speech', 'specialist_3_hate_speech'],
        num_rows: 1120
    })
    test: Dataset({
        features: ['instagram_comments', 'offensive_language', 'offensiveness_levels', 'antisemitism', 'apology_for_the_dictato

In [ ]:
db["train"].features["offensive_language"]

Value(dtype='bool', id=None)

In [ ]:
db["test"][0:3]

{'instagram_comments': ['Mais um lixo',
  'Essa mulher é doente.pilantra!',
  'Vagabunda. Comunista. Mentirosa. O povo chileno nao merece uma desgraça desta.'],
 'offensive_language': [True, True, True],
 'offensiveness_levels': [1, 3, 3],
 'antisemitism': [False, False, False],
 'apology_for_the_dictatorship': [False, False, False],
 'fatphobia': [False, False, False],
 'homophobia': [False, False, False],
 'partyism': [False, False, True],
 'racism': [False, False, False],
 'religious_intolerance': [False, False, False],
 'sexism': [False, False, True],
 'xenophobia': [False, False, False],
 'offensive_&_non-hate_speech': [True, True, False],
 'non-offensive': [False, False, False],
 'specialist_1_hate_speech': [False, False, False],
 'specialist_2_hate_speech': [False, False, False],
 'specialist_3_hate_speech': [False, False, False]}

In [ ]:
# Rename target column to the default name expected by HF models: “label”
db = db.rename_column("offensive_language", "label")

# Drop columns not required for this experiment
db = db.remove_columns(
    [
        "offensiveness_levels",
        "antisemitism",
        "apology_for_the_dictatorship",
        "fatphobia",
        "homophobia",
        "partyism",
        "racism",
        "religious_intolerance",
        "sexism",
        "xenophobia",
        "offensive_&_non-hate_speech",
        "non-offensive",
        "specialist_1_hate_speech",
        "specialist_2_hate_speech",
        "specialist_3_hate_speech"
    ]
)

### Tokenization with BERTimbau

**Model:** BERTimbau (bert‑base‑portuguese‑cased)
[https://huggingface.co/neuralmind/bert-base-portuguese-cased](https://huggingface.co/neuralmind/bert-base-portuguese-cased)

BERTimbau is a BERT model pre‑trained on Brazilian Portuguese.
The corresponding tokenizer must be loaded to pre‑process the dataset’s comment text before feeding it to the model.


In [ ]:
checkpoint = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Create a preprocessing routine that tokenizes the input text with the BERTimbau tokenizer and truncates any sequence that exceeds the model’s maximum input length.

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["instagram_comments"], truncation=True)

In [8]:
tokenized_db = db.map(preprocess_function, batched=True)

Map: 100%|██████████| 1400/1400 [00:00<00:00, 22284.90 examples/s]


### Dynamic Padding with `DataCollatorWithPadding`

`DataCollatorWithPadding` builds mini‑batches from a list of dataset items and applies padding as needed.
With `padding=True` (default), each batch is padded only up to the length of the longest sequence **in that batch**, instead of padding every example in the entire dataset to the model’s global maximum length.
This dynamic padding strategy reduces memory usage and speeds up training/inference.


In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

### Evaluation Setup

* **scikit‑learn:** `sklearn.metrics` supplies a wide range of model‑performance metrics.
* **🤗 evaluate:** a unified metrics API for Hugging Face workflows — see [https://huggingface.co/docs/evaluate/a\_quick\_tour](https://huggingface.co/docs/evaluate/a_quick_tour).


In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    #compute precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    #compute accuracy score
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, 
            "f1": f1,
            "precision": precision,
            "recall": recall
            }

### Model Training

Fine‑tune the pre‑trained BERTimbau model on the training split to classify comments as offensive or non‑offensive.

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

### Optimizer Setup

Define an optimization function that configures a learning‑rate schedule along with key training hyperparameters.

Further details on training a TensorFlow model with Keras:
[https://huggingface.co/docs/transformers/training#train-a-tensorflow-model-with-keras](https://huggingface.co/docs/transformers/training#train-a-tensorflow-model-with-keras)


In [ ]:
batch_size = 16
num_epochs = 1
batches_per_epoch = len(tokenized_db["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

### Model Loading

Load BERTimbau via `TFAutoModelForSequenceClassification`.

`TFAutoModelForSequenceClassification` is a generic wrapper that instantiates the appropriate sequence‑classification model when called with `from_pretrained()` or `from_config()`.
The sequence‑classification head is a linear layer on top of the pooled hidden states produced by the base model (BERTimbau in this case).

Reference: [https://huggingface.co/docs/transformers/main/en/model\_doc/auto#transformers.TFAutoModelForSequenceClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.TFAutoModelForSequenceClassification)


<img src="../figs/transformer-and-head.svg">
https://huggingface.co/course/chapter2/2?fw=tf

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

C:\Users\sergi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Use `prepare_tf_dataset()` to wrap each split as a TensorFlow `tf.data.Dataset`, enabling efficient, streaming‑friendly training and evaluation.


In [14]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_db["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_db["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_db["test"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [15]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108923136 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 108924674 (415.51 MB)
Trainable params: 108924674 (415.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Configure model for training:

In [16]:
model.compile(optimizer=optimizer)

Compute prediction accuracy

In [17]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
callbacks = [metric_callback]

Invoke the fit method to train the model on the training and validation datasets.

In [ ]:
start_time = datetime.now()

history = model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs, callbacks=callbacks)

end_time = datetime.now()
training_time = (end_time - start_time).total_seconds()

280/280 [==============================] - 596s 2s/step - loss: 0.3506 - val_loss: 0.2092 - accuracy: 0.9205 - f1: 0.9205 - precision: 0.9211 - recall: 0.9205


In [19]:
print('Treinamento:')
print('Acurária : {:.1%}'.format(history.history['accuracy'][-1]))
#print('Acurária nos dados de validação: {:.1%}'.format(history.history['val_accuracy'][-1]))
print('Precisão : {:.1%}'.format(history.history['precision'][-1]))
print('Revocação: {:.1%}'.format(history.history['recall'][-1]))
print('F1       : {:.1%}'.format(history.history['f1'][-1]))
print('Tempo de treinamento: {:.1f}s (or {:.1f} minutes)'.format(training_time, training_time/60))

Treinamento:
Acurária : 92.1%
Precisão : 92.1%
Revocação: 92.1%
F1       : 92.1%
Tempo de treinamento: 595.8s (or 9.9 minutes)


In [20]:
dir_save_model = "./"+checkpoint+"-fine-tuned-hatebr-db"
tokenizer.save_pretrained(dir_save_model)
model.save_pretrained(dir_save_model)

### Model Evaluation

Evaluate the model on the test split.

In multi‑class classification, `model.predict` returns a probability vector for each class; `argmax` retrieves the index with the highest probability, yielding the predicted class.


In [21]:
def compute_metrics2(predictions, labels):
    #compute precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    #compute accuracy score
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, 
            "f1": f1,
            "precision": precision,
            "recall": recall
            }

In [ ]:
output_logits = model.predict(tf_test_set).logits
pred_labels = np.argmax(output_logits, axis=-1)
compute_metrics2(pred_labels, db["test"]["label"])

88/88 [==============================] - 35s 378ms/step


{'accuracy': 0.9085714285714286,
 'f1': 0.9084451202066525,
 'precision': 0.9108385868137641,
 'recall': 0.9085714285714286}

In [ ]:
print("Predição: ", pred_labels[:20])
print("Correto:  ", db["test"]["label"][:20])
print("Texto:    ", db["test"]["instagram_comments"][:20])

Predição:  [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
Correto:   [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
Texto:     ['Mais um lixo', 'Essa mulher é doente.pilantra!', 'Vagabunda. Comunista. Mentirosa. O povo chileno nao merece uma desgraça desta.', 'Besta quadrada.', 'Hipocritas', 'Quem tem pena é galinha, mas ela é uma VACA LOUCA.', 'Mande essa Bachelet plantar batata no asfalto. Puta que pariu até o tamborete de bordel se acha no direito de falar do Brasil.', 'Cara de pau.', 'Porque é uma bandida', 'Tudo igual. Só pensam no próprio rabo', 'Cretina!', 'Quem não têm caráter é contra o presidente Bolsonaro', 'Será que alguém em sã consciência ainda vai acreditar nesta farsante? Não acredito em um fio do seu cabelo pintado.', 'Ela tem cara de homem né? Macron deve achar gata', 'Nojo dessa pilantra', 'Bando de canalhas', 'Vieja idiota.', 'Cretina!!!', 'Ridícula nojenta', 'Isso é uma realidade da imbecilidade h

## Prediction

<img src="../figs/transformer-pipeline.svg">
https://huggingface.co/course/chapter2/2?fw=tf

In [ ]:
text = "Sua barraqueira biscoitera"


Tokenize the text and return TensorFlow tensors.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(dir_save_model)
inputs = tokenizer(text, return_tensors="tf")

**Logits** are the raw (unnormalized) prediction vector emitted by the classifier. In multi‑class settings, these logits are typically fed into a softmax layer to obtain a normalized probability distribution—one value per possible class.

Feed the inputs through the model and return the logits:

In [26]:
model = TFAutoModelForSequenceClassification.from_pretrained(dir_save_model)
output_logits = model(**inputs).logits

Some layers from the model checkpoint at ./neuralmind/bert-base-portuguese-cased-fine-tuned-hatebr-db were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./neuralmind/bert-base-portuguese-cased-fine-tuned-hatebr-db.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further

Generate the predicted class from logits:

In [27]:
predicted_class_id = np.argmax(output_logits, axis=-1)[0]
model.config.id2label[predicted_class_id]

'POSITIVE'

### Offensiveness‑Level Classification

The model can be extended to predict the offensiveness level of each comment:

* **Classes:**
  0 = non‑offensive | 1 = slightly offensive | 2 = moderately offensive | 3 = highly offensive
* **Target column:**  `offensiveness_levels`.

##### Data Preprocessing

In [ ]:
db = load_dataset("ruanchaves/hatebr")
db

DatasetDict({
    train: Dataset({
        features: ['instagram_comments', 'offensive_language', 'offensiveness_levels', 'antisemitism', 'apology_for_the_dictatorship', 'fatphobia', 'homophobia', 'partyism', 'racism', 'religious_intolerance', 'sexism', 'xenophobia', 'offensive_&_non-hate_speech', 'non-offensive', 'specialist_1_hate_speech', 'specialist_2_hate_speech', 'specialist_3_hate_speech'],
        num_rows: 4480
    })
    validation: Dataset({
        features: ['instagram_comments', 'offensive_language', 'offensiveness_levels', 'antisemitism', 'apology_for_the_dictatorship', 'fatphobia', 'homophobia', 'partyism', 'racism', 'religious_intolerance', 'sexism', 'xenophobia', 'offensive_&_non-hate_speech', 'non-offensive', 'specialist_1_hate_speech', 'specialist_2_hate_speech', 'specialist_3_hate_speech'],
        num_rows: 1120
    })
    test: Dataset({
        features: ['instagram_comments', 'offensive_language', 'offensiveness_levels', 'antisemitism', 'apology_for_the_dictato

In [ ]:
db["train"].features["offensiveness_levels"]

Value(dtype='int32', id=None)

In [48]:
set(db["train"]["offensiveness_levels"])

{0, 1, 2, 3}

In [ ]:
db["test"][0:5]

{'instagram_comments': ['Mais um lixo',
  'Essa mulher é doente.pilantra!',
  'Vagabunda. Comunista. Mentirosa. O povo chileno nao merece uma desgraça desta.',
  'Besta quadrada.',
  'Hipocritas'],
 'offensive_language': [True, True, True, True, True],
 'offensiveness_levels': [1, 3, 3, 2, 2],
 'antisemitism': [False, False, False, False, False],
 'apology_for_the_dictatorship': [False, False, False, False, False],
 'fatphobia': [False, False, False, False, False],
 'homophobia': [False, False, False, False, False],
 'partyism': [False, False, True, False, False],
 'racism': [False, False, False, False, False],
 'religious_intolerance': [False, False, False, False, False],
 'sexism': [False, False, True, False, False],
 'xenophobia': [False, False, False, False, False],
 'offensive_&_non-hate_speech': [True, True, False, True, True],
 'non-offensive': [False, False, False, False, False],
 'specialist_1_hate_speech': [False, False, False, False, False],
 'specialist_2_hate_speech': [Fal

In [ ]:
db = db.rename_column('offensiveness_levels', 'label')
db = db.remove_columns(
    [
        'offensive_language',
         'antisemitism', 
         'apology_for_the_dictatorship', 
         'fatphobia', 
         'homophobia', 
         'partyism', 
         'racism', 
         'religious_intolerance', 
         'sexism', 
         'xenophobia', 
         'offensive_&_non-hate_speech', 
         'non-offensive', 
         'specialist_1_hate_speech', 
         'specialist_2_hate_speech', 
         'specialist_3_hate_speech'
    ]
)

##### Tokenização pelo Modelo BERTimbau

In [ ]:
checkpoint = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_db = db.map(preprocess_function, batched=True)

Map: 100%|██████████| 1120/1120 [00:00<00:00, 29162.73 examples/s]


##### Treinamento do Modelo

In [ ]:
id2label = {0: "NONE", 1: "MILD", 2: "MODERATE", 3: "SEVERE"}
label2id = {"NONE": 0, "MILD": 1, "MODERATE": 2, "SEVERE": 3}

batch_size = 32
num_epochs = 3
batches_per_epoch = len(tokenized_db["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=total_train_steps*0.1, num_train_steps=total_train_steps)

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4, id2label=id2label, label2id=label2id)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_db["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_db["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_db["test"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [55]:
model.summary()

Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108923136 
                                                                 
 dropout_151 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3076      
                                                                 
Total params: 108926212 (415.52 MB)
Trainable params: 108926212 (415.52 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
model.compile(optimizer=optimizer)
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
callbacks = [metric_callback]

In [ ]:
start_time = datetime.now()
history = model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs, callbacks=callbacks)
end_time = datetime.now()
training_time = (end_time - start_time).total_seconds() 

280/280 [==============================] - 557s 2s/step - loss: 0.9050 - val_loss: 0.7674 - accuracy: 0.6080 - f1: 0.3721 - precision: 0.3748 - recall: 0.4265


C:\Users\sergi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('Treinamento:')
print('Acurária : {:.1%}'.format(history.history['accuracy'][-1]))
print('Precisão : {:.1%}'.format(history.history['precision'][-1]))
print('Revocação: {:.1%}'.format(history.history['recall'][-1]))
print('F1       : {:.1%}'.format(history.history['f1'][-1]))
print('Tempo de treinamento: {:.1f}s (or {:.1f} minutes)'.format(training_time, training_time/60))

Treinamento:
Acurária : 60.8%
Precisão : 37.5%
Revocação: 42.7%
F1       : 37.2%
Tempo de treinamento: 557.1s (or 9.3 minutes)


In [59]:
dir_save_model = "./"+checkpoint+"offensiveness_levels-fine-tuned-hatebr-db"
tokenizer.save_pretrained(dir_save_model)
model.save_pretrained(dir_save_model)

##### Model Evaluation

In [60]:
output_logits = model.predict(tf_test_set).logits
pred_labels = np.argmax(output_logits, axis=-1)
compute_metrics2(pred_labels, db["test"]["label"])

88/88 [==============================] - 39s 409ms/step


C:\Users\sergi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 0.6071428571428571,
 'f1': 0.3638852354021155,
 'precision': 0.38140584411743783,
 'recall': 0.42275632262474366}

##### Prediction

In [75]:
text = ["Seu jaguara corno manso"]

tokenizer = AutoTokenizer.from_pretrained(dir_save_model)
inputs = tokenizer(text, return_tensors="tf")

In [76]:
model = TFAutoModelForSequenceClassification.from_pretrained(dir_save_model)
output_logits = model(**inputs).logits

Some layers from the model checkpoint at ./neuralmind/bert-base-portuguese-casedoffensiveness_levels-fine-tuned-hatebr-db were not used when initializing TFBertForSequenceClassification: ['dropout_151']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./neuralmind/bert-base-portuguese-casedoffensiveness_levels-fine-tuned-hatebr-db.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClass

In [77]:
predicted_class_id = np.argmax(output_logits, axis=-1)[0]
model.config.id2label[predicted_class_id]

'MODERATE'

The current run shows limited convergence. Possible improvements include balancing the training set, tuning hyperparameters, and experimenting with larger or alternative model architectures to increase accuracy.